<a href="https://colab.research.google.com/github/GaneshSelvaraj717/Ganesh-Selvaraj/blob/master/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd # to load dataset
import numpy as np
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords # to get collection of stopwords
from nltk.tokenize import word_tokenize
import string
import gensim
import matplotlib.pyplot as plt

# layers of the architecture
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional

from keras.preprocessing.text import Tokenizer # to encode text to int
from keras.preprocessing.sequence import pad_sequences # to do padding or truncating
from keras.models import Sequential   # the model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# New Section

In [25]:
file = "/corona_nlp_train.csv"
corona_reviews = pd.read_csv(file, encoding = 'latin-1')

In [26]:
corona_reviews.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,62017,Coastal Spain,21-03-2020,Retail analyst Teresa Wickham (of Tunbridge We...,Positive
1,2,57979,NaN,20-03-2020,Prices for display panels are expected to rise...,Positive
2,3,80278,everywhere,06-04-2020,@cryptochrisw Since Covid-19 self quarantine s...,Neutral
3,4,52712,India,18-03-2020,Prices of both\r\r\nKaraknath ( ?)\r\r\nAnd Ka...,Negative
4,5,65192,Everywhere and Nowhere,22-03-2020,Now this is an interesting solution. #handsani...,Extremely Positive


In [27]:
corona_reviews.isnull().values.any()

True

In [29]:
corona_reviews1 = corona_reviews.drop(columns = ['UserName', 'ScreenName', 'Location', 'TweetAt'], axis = 1)

In [30]:
corona_reviews1

,OriginalTweet,Sentiment
0,Retail analyst Teresa Wickham (of Tunbridge We...,Positive
1,Prices for display panels are expected to rise...,Positive
2,@cryptochrisw Since Covid-19 self quarantine s...,Neutral
3,Prices of both\r\r\nKaraknath ( ?)\r\r\nAnd Ka...,Negative
4,Now this is an interesting solution. #handsani...,Extremely Positive
...,...,...
35992,Online grocery shopping has become more like f...,Positive
35993,WVNews247: AG MORRISEY'S OFFICE has received ...,Neutral
35994,We arenÃÂt supposed to be gathering in large...,Neutral
35995,For all those who know food doesnÃÂt come fr...,Extremely Positive


In [31]:
# Data Preprocessing 
def preprocess_text(sen):
    
    sen = re.sub('<.*?>', ' ', sen) # remove html tag

    tokens = word_tokenize(sen)  # tokenizing words

    tokens = [w.lower() for w in tokens]    # lower case

    table = str.maketrans('', '', string.punctuation)  # remove punctuations
    stripped = [w.translate(table) for w in tokens]

    words = [word for word in stripped if word.isalpha()]  # remove non alphabet
    stop_words = set(stopwords.words('english'))

    words = [w for w in words if not w in stop_words]   # remove stop words
    words = [w for w in words if len(w) > 2]  # Ignore words less than 2
    
    return words

In [32]:
sentiment_lines = []
sentences = list(corona_reviews1['Sentiment'])

In [34]:
for sen in sentences:
    # Call the preprocess_text function on each sentence of the review text 
    sentiment_lines.append(preprocess_text(sen))

In [36]:
# Check for the length of the preprocessed text
len(sentiment_lines)

35997

In [37]:
print(sentiment_lines[1])

['positive']


In [38]:
# Now let’s convert the sentiment from string to a binary form of 1 and 0, 
# where 1 is for ‘positive’ sentiment and 0 for ‘negative’.
y = corona_reviews1['Sentiment']
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

In [39]:
EMBEDDING_DIM = 100

# Train word2vec model after preprocessing the reviews
model = gensim.models.Word2Vec(sentences=sentiment_lines, size=EMBEDDING_DIM, window=5, workers=4, min_count=1)

In [40]:
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

Vocabulary size: 4
